In [ ]:
%pip install tensorflow
%pip install tensorflow-datasets
%pip install scipy

In [35]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Flatten,AlphaDropout,Conv2D,Dropout,MaxPool2D
import tensorflow.keras.optimizers 

(train, test) = tfds.load(
    'svhn_cropped',
    split=['train','test'],
    shuffle_files = True,
    as_supervised = True,
)

tf.random.set_seed(42)

In [42]:
train = train.take(2000)
test = test.take(500)

## normalize data [0,1]
def normalize_img(img,label):
    normalized_img = tf.cast(img, tf.float32) / 255
    return normalized_img, label

train_normalized = train.map(normalize_img)
test_normalized = test.map(normalize_img)

train_normalized = train_normalized.shuffle(2000).batch(32)
test_normalized = test_normalized.batch(32)


In [43]:
model = Sequential([
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=(32,32,3)),
    Conv2D(filters=32, kernel_size=(3,3), activation='relu'),
    MaxPool2D(pool_size=(2,2)),
    Conv2D(filters=32, kernel_size=(3,3), activation='relu'),
    Conv2D(filters=32, kernel_size=(3,3), activation='relu'),
    MaxPool2D(pool_size=(2,2)),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(rate=0.3),
    Dense(10, activation='softmax')
])

/Users/daviddevoldpearson/cipher/.venv/lib/python3.13/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
adam_optimizer = tensorflow.keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss='sparse_categorical_crossentropy',optimizer=adam_optimizer,metrics=['accuracy'])
history = model.fit(train_normalized,epochs=15, validation_data=test_normalized)


Epoch 1/15
 5/63 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.1320 - loss: 2.2951  

2025-11-02 21:35:07.056763: I tensorflow/core/kernels/data/tf_record_dataset_op.cc:396] The default buffer size is 262144, which is overridden by the user specified `buffer_size` of 8388608


63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.1915 - loss: 2.2575 - val_accuracy: 0.1840 - val_loss: 2.2629
Epoch 2/15
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.1955 - loss: 2.2465 - val_accuracy: 0.1840 - val_loss: 2.2586
Epoch 3/15
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.1955 - loss: 2.2455 - val_accuracy: 0.1840 - val_loss: 2.2535
Epoch 4/15
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.1955 - loss: 2.2464 - val_accuracy: 0.1840 - val_loss: 2.2467
Epoch 5/15
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.2190 - loss: 2.1727 - val_accuracy: 0.2920 - val_loss: 2.0728
Epoch 6/15
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.3730 - loss: 1.8282 - val_accuracy: 0.4440 - val_loss: 1.6656
Epoch 7/15
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.5360 - loss: 1.3892 - val_accuracy: 0.5880 - val_loss: 1.3102
Epoch 8/15
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.6530 - loss: 1.0843 - val_accuracy: 0.6160 - val_loss: 1.

In [ ]:
test_accuracy = model.evaluate(test_normalized)